# Creating a data set for model training from 2021 season data

In [256]:
#Imports
import statsapi
import datetime 
import pandas as pd
import json

In [257]:
#Getting start date and end date of a season
def season_start_end(year):
    season_info = statsapi.get("seasons", {"sportId": 1, "season": year})
    season_start = season_info['seasons'][0]['regularSeasonStartDate']
    season_end = season_info['seasons'][0]['regularSeasonEndDate']
    return [season_start, season_end]

In [258]:
start_end_2021 = season_start_end(2021)
season_start = start_end_2021[0]
season_end = start_end_2021[1]

In [259]:
#Adding 35 days to the season start, when I want to start collecting data
season_start_offset = (datetime.datetime.strptime(season_start, "%Y-%m-%d") + datetime.timedelta(days=35)).strftime('%Y-%m-%d')

In [260]:
#Geting all season matchup outcomes
def season_match_outcomes(season_start, season_end):
    dict_list = []
    for i in statsapi.schedule(start_date=season_start, end_date=season_end):
        match_dict = {}
        try:
            match_dict['game_id'] = i['game_id']
            match_dict['win'] = i['winning_team']
            match_dict['loss'] = i['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [261]:
#Setting variable to the output for season match outcomes
all_matches = season_match_outcomes(season_start_offset, season_end)

In [262]:
#Creating a list of all game ids for period in 2021 season
game_ids = [i['game_id'] for i in all_matches]

In [ ]:
game_info = statsapi.boxscore_data(632520, timecode=None)
game_info

In [10]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [11]:
#Creating json from above dict_list
#Commenting out to not run again
#with open('roster_dict', 'w') as fout:
#    json.dump(dict_list, fout)

##### Adding date to this, but keeping separate file

In [ ]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['date'] = game_info['gameId'][0:10].replace('/', '-')
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [265]:
#Creating json from above dict_list
#Commenting out to not run again
#with open('roster_date_dict', 'w') as fout:
#    json.dump(dict_list, fout)

## Creating Player Stats function

###### Work in progress, not quite done yet

In [ ]:
#Creation of this function further down -  need to use it in the stats function

#Function to sum values in two dictionaries
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

##### Keeping for reference for now
def player_stats(id,start_date, end_date, season):
    player_info = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    #dict where I will add all the collected stats
    stat_dict = {}
    #Temporary storage for fielding dicts for all positions
    fielding_list = []
    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []
    try:
        player_hitting_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    try: 
        for i in player_info['people'][0]['stats'][1]['splits']:
            if player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'].isin(pos_codes):
                pass
            else:
                pos_codes.append(player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'])
                fielding_list.append(player_info['people'][0]['stats'][i]['splits'][0]['stat']) 
        all_fielding = {}
        for i in fielding_list:
            all_fielding.update(i)

            stat_dict['fielding'] = 1 #Placeholder
    except: pass
    try:
        player_pitching_stats = player_info['people'][0]['stats'][2]['splits'][0]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except:pass
    return stat_dict

In [253]:
#Create function to get player stats
#Inputs are the id, start date and end date

#Currently not getting all the fielding stats, as there are more than one set of them - based on the pos they play - fix in progress

#Updates:
# Need to make 3 API calls, one for hitting, fielding, and pitching.  Might need to add an if statement for pitching/fielding based on position, we will see
# use the add_dicts function to sum all the fielding stats for each position played, as they are split
# Store them all in an organized dictionary for later use

def player_stats(id,start_date, end_date, season):

    #Getting player stats - separately because the API switches their order when fetched at the same time 
    hitting_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    pitching_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    fielding_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[fielding],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})

    #dict where I will add all the collected stats
    stat_dict = {}

    #Temporary storage for fielding dicts for all positions
    fielding_list = []

    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []

    try:
        player_hitting_stats = hitting_stats['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    
    try:
        player_pitching_stats = pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except: pass

    try:
        for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
            if fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'] in pos_codes:
                pass
            else:
                pos_codes.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'])
                fielding_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
                
        all_fielding = {}

        for i in range(len(fielding_list)):
            all_fielding = add_dicts(all_fielding, fielding_list[i])
        stat_dict['fielding'] = all_fielding
        
        
    except:pass
    return stat_dict

In [ ]:
#Testing function for 2022
player_stats(518934,'2022-04-05','2022-08-12', 2022)

In [ ]:
#Testing player stats for a pitcher - Works, but incorrect pitching results, need the last entry, [-1]
player_stats(593974,'2021-04-01', '2021-05-06', 2021)

#### Various tests and examples trying to figure out how to get the above function working

In [ ]:
#Checking length of fielding_stats
fielding_stats = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})
len(fielding_stats['people'][0]['stats'][0]['splits'])

In [ ]:
#Verifying location of position code for looping
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']['position']['code']


In [ ]:
#Verifying position of one player's single position fielding stats
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][1]['splits'][0]['stat']

In [ ]:
#Testing to see if I can combine fielding stats into a dict
#It works except for the 'position' dictionary, which I do not need the info from
#Also combines fielding as a string, but I can fix that later, if I decide to use that stat (thinking I will not for now)
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

            if i == x:

                d3[i]=(j+y)
except: pass
print(d3)

In [ ]:
#Testing for three dictionaries
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d0 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][2]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
except: pass
d4 = dict(d3)
d4.update(d0)
try:
    for i, j in d3.items():

        for x, y in d0.items():

                if (i == x):

                    d4[i]=(j+y)
except: pass
print(d4)

In [87]:
#Making a function to shorten the code - will use this in the stats function to get all the relevant fielding stats added together
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

In [ ]:
#Testing add_dicts starting with an empty dictionary
empty_dict = {}
dict_list = [d1,d0,d2]
for i in range(len(dict_list)):
    empty_dict = add_dicts(empty_dict,dict_list[i])
empty_dict

In [ ]:
#Getting the if statement to work inside the for loop of the player_stats function and 
#creating a list of unique fielding stats for a player (they duplicated for some reason)
temp_list = []
temp_list2 = []
fielding_stats = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})
for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
    if fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'] in temp_list:
        pass
    else:
        temp_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'])
        temp_list2.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
temp_list2

In [ ]:
#Getting add_dicts function to work to combine all fielding stats into one dictionary - strings will be messed up
dict_A = {}
for i in range(len(temp_list2)):
    dict_A =  add_dicts(dict_A, temp_list2[i])
dict_A

## Pulling some stats for a team roster/matchup

In [ ]:
#reading json
f = open('roster_dict')
data = json.load(f)

#### Need to get the date of the game for the player_stats function to work properly - this date will be the end date, with the start of season offset date as the start

In [ ]:
#Looking at the first matchup 

#Getting relevant data from dictionary
matchup_one = data[0]
game_id = matchup_one['game_id']
home_roster = matchup_one['home_roster']
away_roster = matchup_one['away_roster']
date = statsapi.boxscore_data(game_id, timecode=None)['gameId'][0:10].replace('/', '-')

#Pulling stats for each player - had to look up date manually for now

for i in home_roster:
    print(player_stats(i,'2021-04-01', date, 2021))

#Adding stats to a df - to be updated
df = pd.DataFrame()

### Troubleshooting again for player_stats, with pitching specifically
###### Running into issues with pitchers being on the roster but not pitching for the team yet in the season

In [ ]:
#Looking at how to get the total stats for a pitcher in the season so far (As they are often traded and a separate entry is created for each team)
pitching_stats = statsapi.get("people", {"personIds": 593974, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-05-06,season=2021)"})
pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']

In [ ]:
#Looking for Gerrit Cole's stats with function - was returning {} before I fixed the function now
player_stats(543037,'2021-04-01', '2021-05-06', 2021)

In [ ]:
#Looking for Gerrit Cole's stats straight from the API - works fine
statsapi.get("people", {"personIds": 543037, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-05-06,season=2021)"})['people'][0]['stats'][0]['splits'][-1]['stat']